## Libraries

In [8]:
import sys
import os
from requests.auth import HTTPProxyAuth

# Obtener la ruta del directorio actual del notebook
notebook_dir = os.getcwd()

# Añadir el directorio principal 'project' al path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

import config
from config import PROXI_USER, PROXI_PWD, EMAIL, SUBJECT, PROXIES
import pandas as pd

import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

from bs4 import BeautifulSoup
from tqdm import tqdm

from requests_ntlm import HttpNtlmAuth


## Data Collection

### NO USAR

In [2]:
#si se incluye mas de una fecha o serie, incluir /
fecha = '2024-10-04'
serie = 'SF43718'

#url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/"+serie+"/datos/"+fecha+"/"+fecha+""

headers = {
    'Bmx-Token': '80f8b40305f8c5067445bd8868c2be2fd29f526c597fbadffa7cca63509a25a7',  # Cambia esto por el header que necesites
    'Content-Type': 'application/json',
}

proxies = PROXIES
auth = HTTPProxyAuth(PROXI_USER, PROXI_PWD)


### Precio Dollar

In [9]:
API_TOKEN = '80f8b40305f8c5067445bd8868c2be2fd29f526c597fbadffa7cca63509a25a7'
url_banxico = 'https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43787,SF43784/datos/oportuno?token='+API_TOKEN

In [26]:
try:
    #response = requests.get(url,  proxies=proxies, auth=auth, headers=headers)
    response = requests.get(url_banxico)
    
    if response.status_code == 200:
        dataDolar = response.json()
    else:
        print(f"Error en Peticio: {response.status_code}")
        print(f"Detalles del error: {response.text}")
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    #print(f"Error: {response.status_code}, {response.text}")


### Precio Crypto-Monedas

In [25]:
urlCriptos = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=100&page=1&sparkline=false'


In [27]:
try:
    #response = requests.get(url,  proxies=proxies, auth=auth, headers=headers)
    response = requests.get(urlCriptos)
    
    if response.status_code == 200:
        dataCryptos = response.json()
    else:
        print(f"Error en Peticio: {response.status_code}")
        print(f"Detalles del error: {response.text}")
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    #print(f"Error: {response.status_code}, {response.text}")

## Data Understanding

### Dolar

In [29]:
dataDolar.keys()

dict_keys(['bmx'])

### Previo Venta

In [30]:
#Venta
fechaVenta = dataDolar['bmx']['series'][0]['datos'][0]['fecha']
precioVenta = dataDolar['bmx']['series'][0]['datos'][0]['dato']

### Precio Compra

In [31]:
#Compra
#response['bmx']['series'][1]
fechaCompra = dataDolar['bmx']['series'][1]['datos'][0]['fecha']
precioCompra = dataDolar['bmx']['series'][1]['datos'][0]['dato']

### Crypto Monedas

## Data Preparation

### Armado de Mensaje

In [24]:
mensaje = "\nHola, Informacion sobre tipo de Cambio:\nVenta:\n Fecha Venta: {}\n Valor: ${}\nCompra\n Fecha Compra: {}\n Valor: ${}".format(fechaVenta, precioVenta, fechaCompra, precioCompra)
print(mensaje)


Hola, Informacion sobre tipo de Cambio:
Venta:
 Fecha Venta: 08/10/2024
 Valor: $19.3980
Compra
 Fecha Compra: 08/10/2024
 Valor: $19.3890
